<h2>Initialise files</h2>

In [7]:
import pandas as pd
import numpy as np
import math
import os
from functools import reduce

Declare variable constants

In [2]:
FOLER_PATH = 'UOA_raw'
OUTPUT_FILE_PATH = 'uoa_gen'

Parse, combine and output data in all UOA files

In [30]:
import csv

columns_headers = [] 
data = []

for entry in os.listdir(FOLER_PATH):
    file_path = os.path.join(FOLER_PATH, entry)
    
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        column_headers = df.columns.values.tolist()
        data.extend(df.loc[:].values.tolist())

# flatten as single level array 
all_data = []
all_data.extend([column_headers])
all_data.extend(data)

output_file = open(f'{OUTPUT_FILE_PATH}.csv', 'w',  newline='')
writer =  csv.writer(output_file)
writer.writerows(all_data)
output_file.close()

<h2>ETL data</h2>

"""
add vol * mid 
arrange according to ticker / DTE / type
calculate total call + put 
"""